In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from scipy.cluster.hierarchy import linkage,fcluster,dendrogram
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/home-credit-default-risk/application_train.csv')

In [ ]:
prev = pd.read_csv('../input/home-credit-default-risk/previous_application.csv')

In [ ]:
des = pd.read_csv('../input/home-credit-default-risk/HomeCredit_columns_description.csv')

In [ ]:
des[des['Row']=='OWN_CAR_AGE']['Description']

In [ ]:
df.isna().any().sum()

In [ ]:
df.loc[:,((df.isna().sum()/len(df)) > .25)].isna().sum().sort_values(ascending=False)

In [ ]:
df.loc[:,((df.isna().sum()/len(df)) <= .4) & ((df.isna().sum()/len(df)) > 0)].isna().sum().sort_values(ascending=False)

In [ ]:
df.loc[:,((df.isna().sum()/len(df)) <= .25) & ((df.isna().sum()/len(df)) > 0)].isna().sum().sort_values(ascending=False)

In [ ]:
import re

flag1 = re.findall(r'(FLAG_DOCUMENT_\d+)',str(df.columns.to_list()))
flag2 = re.findall(r'(\w+_CNT_SOCIAL_CIRCLE)',str(df.columns.to_list()))
flag3 = re.findall(r"'(\w+_(?:MODE|MEDI|AVG))'",str(df.columns.to_list()))
years = re.findall(r"'(YEARS_BUILD.+?)'",str(df.columns.to_list()))
areas = re.findall(r"'(COMMONAREA.+?)'",str(df.columns.to_list()))

In [ ]:
df[flag1].describe()

In [ ]:
df[flag3].hist(figsize=(25,25))
plt.show()

In [ ]:
des[des['Row'].isin(years)]['Description'][47]

In [ ]:
sns.scatterplot(years[0],areas[0],data=df)

In [ ]:
plt.figure(figsize=(40,30))
sns.heatmap(df.isna().transpose(),
            cmap="YlGnBu",
            cbar_kws={'label': 'Missing Data'})

plt.show()

In [ ]:
des[des['Row'] == 'EXT_SOURCE_2']['Description']

In [ ]:
des[des['Row'] == 'FLAG_DOCUMENT_21']['Description']

In [ ]:
outlier = df[df['OBS_30_CNT_SOCIAL_CIRCLE']>=300].index

In [ ]:
sns.scatterplot('OBS_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE',hue='TARGET',data=df.drop(outlier,axis=0))
plt.show()

In [ ]:
sns.scatterplot('DEF_30_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE',hue='TARGET',data=df.drop(outlier,axis=0))
plt.show()

In [ ]:
sns.scatterplot('OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE',hue='TARGET',data=df.drop(outlier,axis=0))
plt.show()

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(df[['OBS_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE']].corr())
# sns.heatmap(df['Target'])
plt.show()

In [ ]:
df[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']].hist()

In [ ]:
df['OWN_CAR_AGE'].plot(kind='box')

In [ ]:
len(df[(df['OWN_CAR_AGE'] > df['OWN_CAR_AGE'].quantile(.25)) & (df['OWN_CAR_AGE'] < df['OWN_CAR_AGE'].quantile(.75))]['OWN_CAR_AGE'].notnull())/len(df['OWN_CAR_AGE'].notnull())

15% data are inside the box

In [ ]:
df.FLAG_OWN_CAR.value_counts()

In [ ]:
df.OWN_CAR_AGE.isna().sum()

In [ ]:
book = linkage(df.select_dtypes([int,float]).isna().transpose())
plt.figure(figsize=(26,6))
dendrogram(book)

plt.show()

In [ ]:
des[des['Row'] == 'AMT_REQ_CREDIT_BUREAU_QRT']['Description'][120]

In [ ]:
df[['AMT_REQ_CREDIT_BUREAU_YEAR','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT']].hist()

-fill median

-drop car age

-fill na category

-amt req qrt dan month

-fillna EXT SOURCE 

In [ ]:
todrop = re.findall(r"\s'(\w+?_(?:AVG|MODE))'",str(df.columns.to_list())) + ['AMT_REQ_CREDIT_BUREAU_YEAR','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_HOUR','OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OWN_CAR_AGE']

In [ ]:
df_raw = df.copy()

In [ ]:
df = df_raw.drop(todrop,axis=1).copy()

In [ ]:
obj_col = df.select_dtypes('object').columns
num_col = ['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT']

In [ ]:
df[obj_col] = df[obj_col].fillna('None')
df[num_col] = df[num_col].fillna(-1)

In [ ]:
!git clone https://github.com/DmitryUlyanov/Multicore-TSNE.git
!pip install ./Multicore-TSNE/.

In [ ]:
from MulticoreTSNE import MulticoreTSNE as TSNE

tsne = TSNE(n_jobs=-1)

features_tsne = tsne.fit_transform(df.select_dtypes('int','float'))

In [ ]:
feature = pd.DataFrame(features_tsne,columns=['x','y'])

In [ ]:
df_tsne = pd.concat([df.select_dtypes('object').copy(),feature],axis=1)

In [ ]:
for cat in df_tsne.select_dtypes('object').columns:
    plt.figure(figsize=(12,12))
    plt.legend(loc='upper left')
    sns.scatterplot(x='x',y='y',hue=cat,data=df_tsne)
    plt.show()

In [ ]:
# df = df.join(prev,on='SK_ID_CURR',rsuffix='_PREV')

In [ ]:
df_clean = pd.concat([df,pd.get_dummies(df.select_dtypes('object'),drop_first=True)],axis=1).select_dtypes(['int','float']).drop('SK_ID_CURR',axis=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.impute import SimpleImputer
# from sklearn.decomposition import PCA
# from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.model_selection import train_test_split

X = df_clean.copy()
y = X.pop('TARGET')


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.3,stratify=y,random_state=42)

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

impute = SimpleImputer(strategy='median')
scale =  StandardScaler()

X_train = impute.fit_transform(X_train)
X_test = impute.transform(X_test)

X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

In [ ]:
# baseline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_validate

def compare_model(classifier,name='',graph=False):
    
    scoring = {
           'Precision': 'precision_macro',
           'Recall': 'recall_macro',
           'F1':'f1_macro',
           'ROC_AUC'  : 'roc_auc'
    }
    score_dict = cross_validate(classifier, X_train, y_train,
                              cv=10,
                              scoring=scoring)
    
    scores = pd.DataFrame(columns=['Model','Precision','Recall','F1','ROC AUC'])
    scores = scores.append(pd.Series({ 
                                      'Model' : name,
                                      # 'Accuracy' : score_dict['test_Accuracy'].mean(),
                                      'Precision' : score_dict['test_Precision'].mean(),
                                      'Recall' : score_dict['test_Recall'].mean(),
                                      'F1' : score_dict['test_F1'].mean(),
                                      'ROC AUC' : score_dict['test_ROC_AUC'].mean()
    }),ignore_index=True)
 
    return scores

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer


models =  { "Decision Tree" : DecisionTreeClassifier(),
            "Random Forest" : RandomForestClassifier(),
            "XGBoost" : XGBClassifier(), 
            "LogisticRegression" : LogisticRegression(max_iter=1200), 
            "CatBoost" :CatBoostClassifier(verbose=False) ,
            'LGBM' : LGBMClassifier(),
            }

scores = pd.DataFrame(columns=['Model','Precision','Recall','F1','ROC AUC'])
# probability = {}

for model in models:
  scores = scores.append(compare_model(models[model],name=model))

In [ ]:
scores.sort_values('ROC AUC',ascending=False).reset_index(drop=True)

In [ ]:
# space = {'max_depth': hp.quniform('max_depth', 2, 10, 2),'learning_rate': hp.uniform('learning_rate', .001,.9)}

# # Set up objective function
# def objective(params):
#     params = {'max_depth': int(params['max_depth']),'learning_rate': params['learning_rate']}
#     gbm_clf = GradientBoostingClassifier(n_estimators=100, **params) 
#     best_score = cross_val_score(gbm_clf, X_train, y_train, scoring='accuracy', cv=2, n_jobs=4).mean()
#     loss = 1 - best_score
#     return loss

# # Run the algorithm
# best = fmin(fn=objective,space=space, max_evals=20, rstate=np.random.RandomState(42), algo=tpe.suggest)
# print(best)

In [ ]:
# from tpot import TPOTClassifier

# # Assign the values outlined to the inputs
# number_generations = 5
# population_size = 5
# offspring_size = 5
# scoring_function = 'roc_auc'

# # Create the tpot classifier
# tpot_clf = TPOTClassifier(generations=number_generations, population_size=population_size,
#                           offspring_size=offspring_size, scoring=scoring_function,
#                           verbosity=2, random_state=42, cv=2)

# # Fit the classifier to the training data
# tpot_clf.fit(X_train, y_train)

# # Score on the test set
# print(tpot_clf.score(X_test, y_test))

In [ ]:
!pip install optuna

In [ ]:
puter = SimpleImputer()
scaler1 = StandardScaler()
X_change = puter.fit_transform(X)
X_change = scaler1.fit_transform(X)

X = pd.DataFrame(X_change, columns=X.columns)

In [ ]:
from optuna.integration import LightGBMPruningCallback
import optuna  # pip install optuna
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

def objective(trial, X, y):
    param_grid = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="binary_logloss",
            early_stopping_rounds=100,
            callbacks=[
                LightGBMPruningCallback(trial, "binary_logloss")
            ],  # Add a pruning callback
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)

In [ ]:
# study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
# func = lambda trial: objective(trial, X, y)
# study.optimize(func, n_trials=20)

In [ ]:
# print(study.best_params)

In [ ]:
model = LGBMClassifier(n_estimators= 10000,
                       learning_rate= 0.13072382959785664, 
                       num_leaves= 2260, max_depth= 6,
                       min_data_in_leaf= 3500, 
                       lambda_l1= 20, 
                       lambda_l2= 0,
                       min_gain_to_split= 0.485502034262153, 
                       bagging_fraction= 0.7, 
                       bagging_freq= 1, 
                       feature_fraction= 0.5)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
scoring = {
       'Precision': 'precision_macro',
       'Recall': 'recall_macro',
       'F1':'f1_macro',
       'ROC_AUC'  : 'roc_auc'
}
score_dict = cross_validate(model, X_train, y_train,
                          cv=10,
                          scoring=scoring)

scores = pd.DataFrame(columns=['Name','Precision','Recall','F1','ROC AUC'])
scores = scores.append(pd.Series({ 
                                    'Name':'LGBMTuned',
                                  'Precision' : score_dict['test_Precision'].mean(),
                                  'Recall' : score_dict['test_Recall'].mean(),
                                  'F1' : score_dict['test_F1'].mean(),
                                  'ROC AUC' : score_dict['test_ROC_AUC'].mean()
}),ignore_index=True)

In [ ]:
print(scores)

In [ ]:
from sklearn.metrics import f1_score,classification_report

print(classification_report(y_test,model.predict(X_test)))

In [ ]:
from sklearn import metrics
metrics.plot_roc_curve(model, X_test, y_test)